# Ejemplo práctico de un modelo NO supervisado de clasificación
---

Autor: Manuel Díaz Bendito

---

El objetivo de este notebook, dado el dataset [Wine](https://archive.ics.uci.edu/dataset/109/wine), desarrollar un modelo de clusterización no supervisado para ver si somos capaces de identificar los 3 tipos distintos de vino que hay presentes en los datos explorados. Si bien se trata de un dataset etiquetado, es un muy buen ejercicio a llevar a cabo para mostar las virtudes de la reducción de dimensionalidad, de los modelos más modernos de clusterización, y las capacidades de visualización y entendimiento que pueden utilizarse para un análisis posterior.

El desarrollo del notebook se compone de 4 pasos fundamentales:

* La carga, preparación y realización del Exploratory Data Analysis (EDA) de los datos.
* El procesado e ingeniería de características de estos datos.
* El entrenamiento y visualización de la clusterización o segmentación de los datos.
* La validación de los resultados obtenidos, utilizando las etiquetas del dataset, y la conclusión final del desarrollo.

---

# 0 - Instalaciones necesarias e importación de librerías utilizadas

## Instalación de librerías
---
Además de la librería propia del dataset objeto de estudio, se han instalado las librerías de [UMAP](https://umap-learn.readthedocs.io/en/latest/) y de [HBDSCAN](https://pypi.org/project/hdbscan/), que se utilizarán para la segmentación de los datos y su posterior visualización, y que funcionan muy bien juntas.

Si durante la fase de importación, el usuario se encuentra con algún error de importación, deberá añadir la instalación de dicha librería en este bloque.

---

In [ ]:
!pip install ucimlrepo

In [ ]:
!pip install hdbscan

In [ ]:
!pip install umap-learn

In [ ]:
!pip install optuna

## Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import hdbscan
import umap.umap_ as umap
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import random

from ucimlrepo import fetch_ucirepo 

In [ ]:
pd.options.display.max_columns = 100

# 1 - Carga, preparación y EDA de los datos

## 1.1 - Carga del dataset
---
La carga del dataset se realiza utilizando la librería desarrolada por la Universidad Pública de Irvine.

---

In [ ]:
# fetch dataset
wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
df_train_raw = wine.data.features 
df_target_raw = wine.data.targets 


In [ ]:
df_train_raw

## 1.2 - Preparación del dataset
---
En este punto, se unen datos y las etiquetas (descargados por separado en la carga del dataset).

---

In [ ]:
# Imprimimos por pantalla cómo es la variable objetivo inicialmente.
df_target_raw.value_counts()

In [ ]:
# Unimos datos y etiquetas en el mismo dataframe y aplicamos el mapeo de la variable objetivo.
df_train_raw['target'] = df_target_raw['class']

In [ ]:
# Imprimimos por pantalla el nuevo target y su presencia (tanto por 1) en el dataset.
df_train_raw['target'].value_counts(normalize = True)

## 1.3 - EDA
---
En este punto, se realiza el EDA del dataset propuesto. En este desarrollo, se ha dividido el EDA en dos puntos:
* Una exploración preliminar de las variables numéricas.
* Una exploración gráfica de las variables numéricas.

Como punto previo a todo el análisis, las variables sobre las que realizar el EDA son las siguientes (las definiciones son aproximaciones en cuanto al contexto y la documentación disponible):
* Alcohol: porcentaje de alcohol del vino - **numérica**
* Malicacid: concentración de ácido málico (un ácido orgánico natural) - **numérica**
* Ash: cantidad de cenizas (minerales) en el vino después de quemarlo - **numérica**
* Alcalinity_of_ash: alcalinidad de las cenizas (capacidad de neutralizar ácidos) - **numérica**
* Magnesium: contenido de magnesio (en partes por millón) - **numérica**
* Total_phenols: concentración total de compuestos fenólicos (afectan sabor y color) - **numérica**
* Flavanoids: subgrupo de fenoles relacionados al sabor y antioxidantes - **numérica**
* Nonflavanoid_phenols: otros fenoles que no son flavonoides - **numérica**
* Proanthocyanins: tipo de tanino (astringencia y color del vino) - **numérica**
* Color_intensity: intensidad del color del vino - **numérica**
* Hue: tono de color (matiz) del vino - **numérica**
* 0D280_0D315_of_diluted_wines: relación de absorbancia, mide fenoles y calidad - **numérica**
* Proline: cantidad de prolina (aminoácido relacionado con el aroma y sabor del vino) - **numérica**

---

### 1.3.1 - Exploración preliminar de las variables numéricas
---
En este punto nos valemos del método describe para ver las métricas más inmediatas de las variables numérias.
Asimismo, obtenemos la matriz de correlación entre ellas.

---

In [ ]:
df_train_raw.describe()

In [ ]:
df_train_raw.corr(numeric_only = True)

---
La primera lectura de este análisis preliminar nos muestra que, en principio, no hay ninguna variable numérica con valores extraños teniendo en cuenta la información que representa. No hay valores negativos, ni disonantes en ninguno de los campos.

Respecto a la correlación, el target parece guardar especial correlación numérica con `Total_phenols`, `Flavanoids` y `0D280_0D315_of_diluted_wines`. Un análisis posterior con un equipo técnico de laboratorio podría arrojar algo más de información al respecto de este hecho.

---

### 1.3.2 - Exploración gráfica de las variables numéricas
---
Para este punto, y por comodidad, se ha desarrollado la función **plot_num_feat**. Esta función grafica, para las agrupaciones de la característica numérica elegidas por el usuario, el porcentaje de clase 0 y de clase 1 acumulados en un diagrama de barras. Además, superupesto al diagrama de barras, se grafica en un diagrama de líneas el porcentaje de cada categoría sobre el total. Asimismo, si el lector lo desea, se puede mostrar por pantalla un box plot de la característica para ayudar en el análisis.

El código de la función está comentado para su comprensión.

---

In [ ]:
def plot_num_feat(df, feat, bins, print_box = False):
  '''
  Función encargada de mostrar un gráfico con, dado bien un número de
  particiones o los puntos de corte de una variable numérica, el % de clases
  de cada agrupación y su presencia total en el dataset.

  Parámetros de entrada:
    - df: dataframe en el que se encuentran los datos a analizar.
    - feature: nombre de la característica a analizar.
    - bins: valor numérico con el número de particiones o lista con los puntos
    de corte deseados para la variable numérica.
    - print_box: bandera que, si se activa, muestra un diagrama de caja de la
    característica numérica analizada.

  La función no tiene parámetros de retorno.
  '''

  # Creamos un nuevo dataframe con las particiones requeridas por el usuario
  df_binned = df[[feat, 'target']].copy()
  df_binned['bins'] = pd.cut(df[feat], bins=bins)

  # Calculamos la distribución de clases por grupo
  df_grouped = df_binned.groupby('bins')['target'].value_counts(normalize=True).unstack(fill_value=0) * 100

  # Calculamos el % de presencia de cada agrupación
  df_counts = df_binned['bins'].value_counts().sort_index()
  df_grouped['% presencia'] = df_counts / df_counts.sum() * 100

  # Graficamos el diagrama de barras del % de clases de cada agrupación
  ax = df_grouped.drop(columns='% presencia').plot(kind='bar',
                                                   stacked=True,
                                                   figsize=(7,7),
                                                   title='% de clases en cada grupo de la característica ' + feat)

  # Graficamos el diagrama de línea del % de presencia de cada agrupación
  df_grouped['% presencia'].plot(kind='line', color='red', linewidth=1.5, linestyle='-.', marker='o', ax=ax, rot=90)

  ax.set_xlabel('Grupos de la característica ' + feat)
  ax.set_ylabel('%')
  ax.legend(['% presencia'] + [f'% clase {c}' for c in df_grouped.drop(columns='% presencia').columns])

  plt.show()

  # Si el usuario lo desea, mostramos un diagrama de caja de la característica
  if print_box:
    df[feat].plot(kind='box')


#### Alcohol
---
La variable alcohol muestra un patrón claro: a medida que aumenta la graduación alcohólica del vino, crece de forma progresiva la proporción de muestras de la clase 1, mientras que las clases 2 y 3 dominan en los niveles más bajos de alcohol. No se observan valores atípicos extremos en el diagrama de caja, pero existe una leve asimetría positiva (hacia valores altos).

Concretamente, a partir de los 13.5 grados, la clase 1 empieza a ser mayoritaria, y a partir de 14 grados prácticamente monopoliza el conjunto.

De esta forma, mantendremos esta variable tal cual, pues su capacidad discriminativa es alta.

---

In [ ]:
plot_num_feat(df_train_raw, 'Alcohol', bins = 10)

#### Malicacid
---
La variable malic acid presenta una distribución amplia y dispersa, con bastantes valores extremos hacia niveles altos. Los vinos con valores bajos (menores a 2) tienden a pertenecer a la clase 1 y 2, mientras que los valores altos reflejan claramente la predominancia de la clase 3.

No se observa un comportamiento estrictamente lineal, sino más bien zonas estables de predominio leve de ciertas clases.

---

In [ ]:
plot_num_feat(df_train_raw, 'Malicacid', 5)

#### Ash
---
La variable ash presenta una distribución compacta, sin valores atípicos fuertes. Se aprecia que los valores más bajos (1.3-2.0) favorecen levemente a la clase 2, pero las diferencias no son muy marcadas en valore superiores a 2.

La presencia de clases es bastante homogénea a través de todos los valores superiores a 2, lo que indica una baja capacidad discriminativa individual.

---

In [ ]:
plot_num_feat(df_train_raw, 'Ash', 5)

#### Alcalinity_of_ash
---
Alcalinity of ash tiene un rango de valores amplio y muestra una ligera tendencia: los valores bajos (10-15) son más frecuentes en la clase 1, mientras que valores intermedios (15-25) no discriminan claramente. Existen algunos valores atípicos hacia valores altos. A partir del 25, se puede ver una evidente predominancia de la clase 2.

No se observa una relación lineal, sino más bien una separación parcial en los extremos.

---

In [ ]:
plot_num_feat(df_train_raw, 'Alcalinity_of_ash', 10)

#### Magnesium
---
La variable magnesium tiene una distribución razonablemente simétrica, aunque con algunos valores elevados considerados atípicos.

Los niveles bajos (70-100) presentan casi toda la muestra de la clase 3, mientras que los intermedios (80-150) son los que prevalencen para la clas 1. La clase 2 se ve distribuida casi uniformemente a lo largo de toda la variable.

---

In [ ]:
plot_num_feat(df_train_raw, 'Magnesium', 5)

#### Total_Phenols
---
Total phenols muestra una relación clara: vinos con altos niveles de fenoles totales (mayores a 2.5) tienden a pertenecer abrumadoramente a la clase 1.
Los valores bajos (menores a 1.5) favorecen clases 2 y 3.

No se detectan outliers preocupantes.

---

In [ ]:
plot_num_feat(df_train_raw, 'Total_phenols', 5)

#### Flavanoids
---
La variable flavanoids presenta un comportamiento muy similar al de total phenols, aunque de forma aún más marcada.

Niveles altos (más de 3) se asocian casi exclusivamente a clase 1.

---

In [ ]:
plot_num_feat(df_train_raw, 'Flavanoids', 5)

#### Nonflavanoid_phenols
---
Nonflavanoid phenols tiene un rango muy estrecho y una distribución menos discriminativa. Sin embargo, valores elevados (>0.5) tienden a asociarse ligeramente a clase 2 y 3.

Los niveles intermedios y bajos no permiten separación clara.

---

In [ ]:
plot_num_feat(df_train_raw, 'Nonflavanoid_phenols', 5)

#### Proanthocyanins
---
La variable proanthocyanins muestra una clara separación entre las clases 1 y 3 a partir de valores rondando el 2.

La clase 2 está presente uniformemente casi en la totalidad de la muestra.

---

In [ ]:
plot_num_feat(df_train_raw, 'Proanthocyanins', 5)

#### Color_intensity
---
Color intensity presenta una separación evidente: vinos con coloraciones más intensas (mayores a 6) son mayoritariamente de la clase 3.

La clase 1 domina en valores intermedios, mientras que la clase 2 predomina en colores menos intensos (<3).

---

In [ ]:
plot_num_feat(df_train_raw, 'Color_intensity', 5)

#### Hue
---
La variable Hue presenta un comportamiento similar al de la coloración, pero intercambiando la clase 3 por la 2, y manteniendo el comportamiento de la clase 1.

---

In [ ]:
plot_num_feat(df_train_raw, 'Hue', 5)

#### 0D280_0D315_of_diluted_wines
---
La variable OD280/OD315 es muy buena discriminadora: valores altos (por encima de 2.8) concentran la mayoría de la clase 1, mientras que valores bajos están más asociados a clase 3.

---

In [ ]:
plot_num_feat(df_train_raw, '0D280_0D315_of_diluted_wines', 5)

#### Proline
---
La variable proline destaca como la mejor separadora: niveles bajos (menos de 600) son casi exclusivos de clase 2, intermedios de clase 3, y niveles muy altos (>1200) son dominio absoluto de la clase 1.

---

In [ ]:
plot_num_feat(df_train_raw, 'Proline', 5)

# 2 - Procesado e ingeniería de características
---
En este punto, se realiza el procesado previo a la ingeniería de características, y la ingeniería de características propiamente dicha. Esta ingeniería de características se ha realizado utilizando la información y los insights obtenidos del EDA (del que se ha decidido mantener todas las variables), y de la tipología del modelo de ML a utilizar, por el cual se han tomado las siguientes decisiones:

* Se rellenarán todos aquellos NaNs presentes en el dataset con la media de cada columna (HDBSCAN no trabaja con NaNs)

* Se estandarizarán todas las variables, para que cuando se haga la reducción de la dimensionalidad, mantengamos todas las variables en el mismo rango de trabajo, quitando outliers y paliando valores atípicos.

---

In [ ]:
# Copiamos el dataset original para no realizar transformaciones sobre este.
df_train = df_train_raw.copy()

## 2.1 - Procesado de los datos e ingeniería de características.

In [ ]:
# Declaramos las características a utilizar en el entrenamiento del modelo, así
# como el target del mismo.
feats2use = [
 'Alcohol',
 'Malicacid',
 'Ash',
 'Alcalinity_of_ash',
 'Magnesium',
 'Total_phenols',
 'Flavanoids',
 'Nonflavanoid_phenols',
 'Proanthocyanins',
 'Color_intensity',
 'Hue',
 '0D280_0D315_of_diluted_wines',
 'Proline',
 'target'
]

In [ ]:
# Realizamos la selección de las características a utilizar del dataset
# procesado
df_FM_train = df_train[feats2use].copy()

In [ ]:
# Definimos las columans con las características, eliminando la columan target.
columns_x = df_FM_train.columns.drop('target')

In [ ]:
# Definimos las variables X e y, con los datos de entrenamiento y la variable
# objetivo, respectivamente.
X = df_FM_train[columns_x]
y = df_FM_train['target']

In [ ]:
# Rellenar NaN con la media de cada columna
X = df_train.fillna(X.mean())

# Estandarizar las variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3 - Modelado
---
Una vez construido el dataset, se procede al entrenamiento de un modelo de Machine Learning.

Con todo lo aprendido anteriormente de los datos, tratándose de un **problema de clasificación binaria**, con un dataset **desbalanceado**, un pequeño grupo de características a utilizar y claras agrupaciones que pueden ayudar en la decisión de la clase final, se ha decidido optar por un **XGBoost** como modelo para este proyecto.

La decisión se ha tomado por la demostrada eficiencia de estos modelos en los problemas de clasificación binaria, en el tratamiento de datasets desbalanceados y en la capacidad de tratamiento de datos nulos como información desconocida.

---

In [ ]:
def draw_umap(features_np, min_dist=0, n_neighbours=50, n_components=2):
    np.random.seed(SEED)
    umap_reducer = umap.UMAP(
        n_components=n_components,
        min_dist = min_dist,
        n_neighbors = n_neighbours,
        metric='euclidean',
        # random_state=SEED
    )
    data_umap = umap_reducer.fit_transform(features_np)

    # Visualización de resultados de UMAP
    if n_components == 2:
        plt.figure(figsize=(10, 7))
        plt.scatter(data_umap[:, 0], data_umap[:, 1], s=1, alpha=0.7)
        plt.xlabel('Componente 1')
        plt.ylabel('Componente 2')
    else:
        # Crear una figura 3D
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')

        # Dibujar el scatter plot
        ax.scatter(data_umap[:, 0], data_umap[:, 1], data_umap[:, 2], s=1, alpha=0.7)

        # Etiquetas de los ejes
        ax.set_xlabel('Componente 1')
        ax.set_ylabel('Componente 2')
        ax.set_zlabel('Componente 3')

        # Definir una vista específica (ángulo de elevación y azimut)
        ax.view_init(elev=30, azim=120)

    plt.title(f'Visualización de datos reducidos con UMAP (min_dist = {min_dist}, n_neighbours = {n_neighbours})')
    # Mostrar el gráfico
    plt.show()

    return data_umap

## 3.0 - Proyección del dataset en 2 dimensiones con UMAP

In [ ]:
# Definimos los parámetros iniciales de UMAP
MIN_DIST = 0.01
N_NEIGH = 45
#N_NEIGH = 50
N_COMPONENTS = 2
SEED = 1234

df_umap = pd.DataFrame()

In [ ]:
print("Calculando UMAP")

# Calcular el resultado de UMAP y guardarlo
umap_result = draw_umap(X_scaled, MIN_DIST, N_NEIGH, N_COMPONENTS)

## 3.1 - Entrenamiento del modelo HDBSCAN

In [ ]:
def objective(trial):
    # Sugerir hiperparámetros
    mc = trial.suggest_int('min_cluster_size', 15, 500)
    ms = trial.suggest_int('min_samples', 15, 250)

    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=mc, 
        min_samples=ms,
        metric='euclidean',
        cluster_selection_method='eom',
        gen_min_span_tree=True,
    )
    
    # Ajustar y predecir clusters
    cluster_labels = clusterer.fit_predict(umap_result)
    
    # Filtrar puntos no ruidosos
    valid_indices = cluster_labels != -1
    cluster_labels_valid = cluster_labels[valid_indices]

    # Evitar configuración con pocos o demasiados clusters
    num_clusters = len(set(cluster_labels_valid))

    # Calcular el validity
    score = clusterer.relative_validity_
    del clusterer
    
    #if (num_clusters == 9):
    #    return score + 0.1
    if (num_clusters < 7) or (num_clusters > 10):
        return score - 0.3
    
    return score  # Maximizar la métrica

# Ejecutar optimización
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=500)

# Mejores parámetros
print("Mejores parámetros:", study.best_params)

In [ ]:
# En base a la exploración anterior, definimos los mejores hiperparámetros del HDBSCAN
best_min_cluster_size = 33
best_min_samples = 36

In [ ]:
# Aplicamos HDBSCAN a nuestro dataframe de umap
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=best_min_cluster_size, 
    min_samples=best_min_samples,
    gen_min_span_tree=True
)

cluster_labels = clusterer.fit_predict(umap_result)

# Añadir `umap_result` y `cluster_labels` al DataFrame original utilizando el índice
df_umap['cluster'] = cluster_labels

print(f"Número de clusters {df_umap[df_umap['cluster'] != -1]['cluster'].nunique()}")
print(f"Validity score: {clusterer.relative_validity_:.2}")

X = np.array(umap_result)

# Filtrar los puntos que no son ruido
valid_indices = cluster_labels != -1 
X_valid = X[valid_indices]
cluster_labels_valid = cluster_labels[valid_indices]

## 3.2 - Uso de UMAP para proyectar los clústers generados

In [ ]:
unique_clusters = np.unique(cluster_labels_valid)
cluster_colors = {
    0 : "#1f77b4",  # azul
    1 : "#ff7f0e",  # naranja
    2 : "#2ca02c",  # verde
}


valid_umap_result = umap_result[cluster_labels != -1]

# Visualizamos los clusters
plt.figure(figsize=(8, 6))

point_colors = np.array([cluster_colors[label] if label in cluster_colors else "#d3d3d3" for label in cluster_labels_valid])

if N_COMPONENTS == 2:
    scatter = plt.scatter(valid_umap_result[:, 0], valid_umap_result[:, 1], s=1, c=point_colors)
    plt.xlabel('Componente 1')
    plt.ylabel('Componente 2')
else:
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Dibujar el scatter plot
    scatter = ax.scatter(valid_umap_result[:, 0], valid_umap_result[:, 1], valid_umap_result[:, 2], c=cluster_labels[cluster_labels != -1], s=1, cmap=cmap)

    # Etiquetas de los ejes
    ax.set_xlabel('Componente 1')
    ax.set_ylabel('Componente 2')
    ax.set_zlabel('Componente 3')

    # Definir una vista específica (ángulo de elevación y azimut)
    ax.view_init(elev=30, azim=160)

plt.title('Visualización de clusters con HDBSCAN')

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=8) 
           for label, color in cluster_colors.items()]
labels = [f'Clúster {label}' if label != -1 else "Ruido" for label in cluster_colors.keys()]
plt.legend(handles, labels, title="Clúster", loc="center left", bbox_to_anchor=(1.05, 0.5), ncol=1)

# Añadir la leyenda
plt.show()

In [ ]:
# Vemos cuantos puntos han sido clasificados como ruido
n_noise = list(cluster_labels).count(-1)
print(f"El número de puntos clasificados como ruido es: {n_noise}")
print(f"lo que representa un {n_noise/len(cluster_labels)*100:.2f}% del total de puntos")

# 4 - Evaluación del modelo obtenido
---
En este último punto, procedemos a evaluar el modelo entrenado. Para ello, cabe resaltar que las métricas para modelos de clusterización son referenciales para varios entrenamientos sobre el mismo conjunto de datos, y una evaluación funcional y gráfica suele dar una mejor visión de la virtud del funcionamiento del modelo. 

---

In [ ]:
df_metrics = df_target_raw.copy()

In [ ]:
df_metrics['prediction'] = list(cluster_labels.astype(list))

In [ ]:
df_metrics['prediction'] = df_metrics['prediction'].map({-1: -1, 0: 1, 1: 2, 2: 3})

In [ ]:
cm = confusion_matrix(df_metrics['class'], df_metrics['prediction'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

print()